In [149]:
#Função para carregar a imagem
def carregar_imagem(caminho):
    with open(caminho, "rb") as f:
        return bytearray(f.read())

In [150]:
def salvar_imagem(caminho, conteudo):
    with open(caminho, "wb") as f:
        f.write(bytearray(conteudo))

In [151]:
# Função para converter imagem colorida para tons de cinza
def converter_para_cinza(imagem):
    # Preservar o cabeçalho estendido (138 bytes neste caso)
    offset_pixels = 138
    cabecalho = imagem_colorida[:offset_pixels]
    largura = 512
    altura = 512

    # Pixels começam após o cabeçalho
    pixels = imagem_colorida[offset_pixels:]

    # Nova imagem com cabeçalho intacto
    nova_imagem = bytearray(cabecalho)

    # Cada linha de pixels deve ser múltiplo de 4 bytes (adicionar padding)
    padding = (4 - (largura * 3) % 4) % 4

    # Processar os pixels RGB para tons de cinza
    for y in range(altura):
        linha = []
        for x in range(largura):
            i = (y * (largura * 3 + padding)) + (x * 3)
            b, g, r = pixels[i:i+3]  # BMP usa ordem BGR
            cinza = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            linha.extend([cinza, cinza, cinza])  # R=G=B
        # Adicionar linha processada à nova imagem
        nova_imagem.extend(linha)
        nova_imagem.extend([0] * padding)  # Padding no final de cada linha

    # Atualizar tamanho do arquivo no cabeçalho
    tamanho_arquivo = len(nova_imagem)
    nova_imagem[2:6] = tamanho_arquivo.to_bytes(4, "little")

    return nova_imagem

In [152]:
# Função para binarizar a imagem em preto e branco
def binarizar_com_cabecalho_estendido(imagem_cinza, limiar=128):
    # Preservar o cabeçalho estendido (138 bytes neste caso)
    offset_pixels = 138
    cabecalho = imagem_cinza[:offset_pixels]
    largura = 512
    altura = 512
    
    # Pixels começam após o cabeçalho
    pixels = imagem_cinza[offset_pixels:]
    
    # Nova imagem com cabeçalho intacto
    nova_imagem = bytearray(cabecalho)
    
    # Cada linha de pixels deve ser múltiplo de 4 bytes (adicionar padding)
    padding = (4 - (largura * 3) % 4) % 4
    
    # Processar os pixels de tons de cinza para preto e branco
    for y in range(altura):
        linha = []
        for x in range(largura):
            i = (y * (largura * 3 + padding)) + (x * 3)
            cinza = pixels[i]  # R=G=B, pois já está em tons de cinza
            # Binarizar com base no limiar
            valor_binario = 255 if cinza >= limiar else 0
            linha.extend([valor_binario, valor_binario, valor_binario])  # R=G=B
        
        # Adicionar linha processada à nova imagem
        nova_imagem.extend(linha)
        nova_imagem.extend([0] * padding)  # Padding no final de cada linha
    
    # Atualizar tamanho do arquivo no cabeçalho
    tamanho_arquivo = len(nova_imagem)
    nova_imagem[2:6] = tamanho_arquivo.to_bytes(4, "little")
    
    return nova_imagem

In [153]:
# Caminho da imagem original e saída
caminho_original = 'images/lena-colorida.bmp'
caminho_saida_cinza = 'images/lena-cinza.bmp'
caminho_saida_binaria = 'images/lena-binaria.bmp'
    

In [154]:
imagem_colorida = carregar_imagem(caminho_original)

In [155]:
# Converter para tons de cinza
imagem_cinza = converter_para_cinza(imagem_colorida)
salvar_imagem(caminho_saida_cinza, imagem_cinza)
print("Imagem convertida para tons de cinza e salva em:", caminho_saida_cinza)

Imagem convertida para tons de cinza e salva em: images/lena-cinza.bmp


In [156]:
# Converter para imagem binarizada
imagem_binaria = binarizar_com_cabecalho_estendido(imagem_cinza, limiar=128)
salvar_imagem(caminho_saida_binaria, imagem_binaria)
print("Imagem binarizada e salva em:", caminho_saida_binaria)

Imagem binarizada e salva em: images/lena-binaria.bmp
